# Python Notebook for Easy Implementation of Isoclassify Direct Method

Author: Aniket Sanghi

Institution: The University of Texas at Austin

This is a general purpose python notebook to work with the isoclassify package to extract stellar parameters using photometry. As the user, you just have to specify the filepaths on your system (corresponding to the location of your files and installations) and the names of the columns in your FITS file. Follow the comments starting with USER. Generally, users will just have to modify cell 1.1 and run subsequent cells as is.

Pre-Requisite: Installation of Isoclassify from https://github.com/danxhuber/isoclassify

Note: The program will NOT crash if your data has NaN values.

## User Inputs: ACTION REQUIRED

In [ ]:
# USER: If no data is available leave the assignment as is.

# USER: Enter your system paths to the FITS file and dust map folder installation
fits_filepath = 'test_data_isoclassify.fits'
dust_map_path = 'your_path/dust_maps'

# USER: If you have defined a subset for your data that is a boolean column and you wish to use those datapoints only. 
# Enter the name of the column and set flag to 1
bool_subset_col = ''
bool_subset_flag = 0

# USER: Enter the names of the columns storing the respective data in your FITS File
# If there are multiple columns for the same photometric band (arising from cross-matches etc.),
# specify names of additional columns in '' separated by commas
# The code will utilize the column that has photometry values and ignore ones that do not.
# NOTE: Multiple column capability is not available for RA, Dec, parallax, and parallax error
# NOTE: If you are specifying a photometric band column but do not have an error column, 
# add '' to the error column name.

# Positional Data Columns
ra_col = 'ra'                  # decimal degrees
dec_col = 'dec'                # decimal degrees
plx_col = 'parallax'           # arcseconds
plx_err_col = 'parallax_error' # arcseconds

# Gaia Photometry
Gmag_col = ['phot_g_mean_mag']
Gmag_err_col = ['phot_g_mean_mag_error']
BPmag_col = ['phot_bp_mean_mag']
BPmag_err_col = ['phot_bp_mean_mag_error']
RPmag_col = ['phot_rp_mean_mag']
RPmag_err_col = ['phot_rp_mean_mag_error']

# 2MASS or Equivalent Photometry
Jmag_col = ['J_2MASS']
Jmag_err_col = ['J_2MASS_err']
Hmag_col = ['H_2MASS']
Hmag_err_col = ['H_2MASS_err']
Kmag_col = ['K_2MASS']
Kmag_err_col = ['K_2MASS_err']

# PAN-STARRS or Equivalent Photometry
gmag_col = ['g_P1']
gmag_err_col = ['g_P1_err']
rmag_col = ['r_P1']
rmag_err_col = ['r_P1_err']
imag_col = ['i_P1']
imag_err_col = ['i_P1_err']
zmag_col = ['z_P1']
zmag_err_col = ['z_P1_err']

# Optical Band Photometry
bmag_col = []
bmag_err_col = []
vmag_col = []
vmag_err_col = []

btmag_col = []
btmag_err_col = []
vtmag_col = []
vtmag_err_col = []

# USER: Enter column names for spectroscopic and seismic data.
# Isoclassify works in absence of this data if you do not have it.

# Spectroscopy (Effective temperature, surface gravity, metallicity)
teff_col = []
teffe_col = []
logg_col = []
logge_col = []
feh_col = []
fehe_col = []

# Seismology (Numax, delta nu)
numax_col = []
numaxe_col = []
dnu_col = []
dnue_col = []

# Dustmap to be used. Options:
# 'allsky' - For Combined19 map
# 'green19' - For Bayestar19 map
# 'none' - No reddening map, fit for Av
# 'zero' - Do not assume reddening

dustmap = 'zero'

# Switches
# USER: Change plot to 1 if you would like isoclassify to plot parameter
# distributions, change print_bolometric_luminosities, print_bolometric_luminosities_errors, print_mabs
# and print_teffs to 1 to print those

plot = 0
print_bolometric_luminosities = 1
print_bolometric_luminosities_errors = 1
print_teffs = 0
print_mabs = 0

## Data Addition Functions

In [ ]:
# USER: Do not modify contents. Run as is.

# Add photometry function
def isoclassify_direct_photometry(photometry_cols, photometry_dict):
    
    band = []
    band_flag = 0
    keys = list(photometry_dict.keys())
    no_band = ['bpmag', 'rpmag', 'gmag', 'zmag']
    
    for counter in range(0, len(photometry_cols), 2):
        
        photo_flag = 0
        
        for ctr in range(0, len(photometry_cols[counter])):
            
            if(photo_flag == 0):
                
                if(photometry_cols[counter][ctr] != '' and ~np.isnan(data[index][photometry_cols[counter][ctr]])):

                    if(not (keys[counter] in no_band)):
                        band.append(keys[counter])

                    photometry_dict[keys[counter]] = data[index][photometry_cols[counter][ctr]]
                    if(photometry_cols[counter+1][ctr] != ''):
                        photometry_dict[keys[counter+1]] = data[index][photometry_cols[counter+1][ctr]]
                    
                    photo_flag = 1
            
            else:
                break
            
    if(len(band)):
        band_flag = 1
        
    return photometry_dict, band, band_flag

# Add spectroscopy function
def isoclassify_direct_spectroscopy(spec_cols, spec_dict):
    
    keys = list(spec_dict.keys())
    
    for counter in range(0, len(spec_cols), 2):
        
        for ctr in range(0, len(spec_cols[counter])):

            if(spec_cols[counter] != '' and ~np.isnan(data[index][spec_cols[counter]])):

                spec_dict[keys[counter]] = data[index][spec_cols[counter]]
                if(spec_cols[counter+1] != ''):
                    spec_dict[keys[counter+1]] = data[index][spec_cols[counter+1]]

    return spec_dict

# Add sesimology function
def isoclassify_direct_seismology(seis_cols, seis_dict):
    
    keys = list(seis_dict.keys())
    
    for counter in range(0, len(seis_cols), 2):
        
        for ctr in range(0, len(seis_cols[counter])):
            
            if(seis_cols[counter] != '' and ~np.isnan(data[index][seis_cols[counter]])):

                seis_dict[keys[counter]] = data[index][seis_cols[counter]]
                if(seis_cols[counter+1] != ''):
                    seis_dict[keys[counter+1]] = data[index][seis_cols[counter+1]]

    return seis_dict

## Package Set-up

In [ ]:
# USER: Do not modify contents. Run as is.

%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os 
import h5py
import numpy as np
import astropy.units as units
from astropy.io import fits
from astropy.table import Table

# Define Environment Variable to Dust Maps
os.environ['DUST_DIR'] = dust_map_path

from isoclassify import DATADIR
from isoclassify.direct import classify as classify_direct
from isoclassify.extinction import query_dustmodel_coords

# Define path to Bolometric Correction Grid
fn = os.path.join(DATADIR,'bcgrid.h5')
bcmodel = h5py.File(fn,'r', driver='core', backing_store=False)

# Open FITS file
hdu_list = fits.open(fits_filepath, memmap=True)
data = hdu_list[1].data
hdu_list.close()

if(bool_subset_flag):
    data = data[data[bool_subset_col] == True]
    
# Output Parameters
lum = []
teff = []
mabs = []
lum_err_pos = []
lum_err_neg = []

## Isoclassify

In [ ]:
# USER: Do not modify contents. Run as is.

for index in range(0, len(data)):
    
    # Index printed for reference
    print(f'Index: {index}')
    
    # Variable initializations
    ra = -99
    dec = -99
    plx = -99
    plx_err = -99
    
    # List of photometry column names
    photometry_cols = [Gmag_col, Gmag_err_col, BPmag_col, BPmag_err_col, RPmag_col, RPmag_err_col, 
                   Jmag_col, Jmag_err_col, Hmag_col, Hmag_err_col, Kmag_col, Kmag_err_col, 
                   gmag_col, gmag_err_col, rmag_col, rmag_err_col, 
                   imag_col, imag_err_col, zmag_col, zmag_err_col, 
                   bmag_col, bmag_err_col, vmag_col, vmag_err_col, btmag_col, btmag_err_col, 
                   vtmag_col, vtmag_err_col]
    
    # List of spectroscopy column names
    spec_cols = [teff_col, teffe_col, logg_col, logge_col, feh_col, fehe_col]
    
    # List of sesimology column names
    seis_cols = [numax_col, numaxe_col, dnu_col, dnue_col]
    
    # Initialized dictionary of photometry variables
    photometry_dict = {'gamag': -99, 'gamage': -99, 'bpmag': -99, 'bpmage': -99, 'rpmag': -99, 'rpmage': -99, 
                       'jmag': -99, 'jmage': -99, 'hmag': -99, 'hmage': -99, 'kmag': -99, 'kmage': -99, 
                       'gmag': -99, 'gmage': -99, 'rmag': -99, 'rmage': -99, 'imag': -99, 'image': -99,
                       'zmag': -99, 'zmage': -99, 'bmag': -99, 'bmage': -99, 'vmag': -99, 'vmage': -99, 
                       'btmag': -99, 'btmage': -99, 'vtmag': -99, 'vtmage': -99}
    
    # Initialize dictionary of spectroscopy and seismology dictionaries
    spec_dict = {'teff': -99, 'teffe': -99, 'logg': -99, 'logge': -99, 'feh': -99, 'fehe': -99}
    seismo_dict = {'numax': -99, 'numaxe': -99, 'dnu': -99, 'dnue': -99}

    # Flag to check if band is defined
    band_flag = 0
    
    # Add RA and Dec
    x = classify_direct.obsdata()
    x.addcoords(data[index][ra_col], data[index][dec_col])
    
    # Add parallax in arcseconds
    x.addplx(data[index][plx_col]/1000,data[index][plx_err_col]/1000)
          
    # Function to read all photometry
    photometry_dict, band, band_flag = isoclassify_direct_photometry(photometry_cols, photometry_dict)
     
    # Function to read all spectroscopy
    spec_dict = isoclassify_direct_spectroscopy(spec_cols, spec_dict)
     
    # Function to read all seismology
    seismo_dict = isoclassify_direct_seismology(seis_cols, seismo_dict)
    
    # Add photometry
    x.addgriz([photometry_dict['gmag'], photometry_dict['rmag'], photometry_dict['imag'], photometry_dict['zmag']], 
              [photometry_dict['gmage'], photometry_dict['rmage'], photometry_dict['image'], 
               photometry_dict['zmage']])
    
    x.addjhk([photometry_dict['jmag'], photometry_dict['hmag'], photometry_dict['kmag']], 
             [photometry_dict['jmage'], photometry_dict['hmage'], photometry_dict['kmage']])
    
    x.addgaia([photometry_dict['gamag'], photometry_dict['bpmag'], photometry_dict['rpmag']], 
              [photometry_dict['gamage'], photometry_dict['bpmage'], photometry_dict['rpmage']])
    
    x.addbv([photometry_dict['bmag'], photometry_dict['vmag']], 
            [photometry_dict['bmage'], photometry_dict['vmage']])
    
    x.addbvt([photometry_dict['btmag'], photometry_dict['vtmag']], 
            [photometry_dict['btmage'], photometry_dict['vtmage']])
    
    # Add spectroscopy
    x.addspec([spec_dict['teff'], spec_dict['logg'], spec_dict['feh']], 
              [spec_dict['teffe'], spec_dict['logge'], spec_dict['fehe']])
    
    # Add seismology
    x.addseismo([seismo_dict['numax'], seismo_dict['dnu']], [seismo_dict['numaxe'], seismo_dict['dnue']])
    
    # Dustmodel extraction
    dustmodel, ext = query_dustmodel_coords(x.ra, x.dec, dustmap)
    
    # Run Isoclassify Direct and test all bands
    for counter, test_band in enumerate(band):  
        if(counter != 0):
            if(paras.lum == 0):
                x.addmag([photometry_dict[test_band]], [photometry_dict[test_band + 'e']])
                paras = classify_direct.stparas(x, bcmodel=bcmodel, dustmodel=dustmodel, band=test_band, ext=ext, plot=plot)
        else:
            if(band_flag == 0):
                print('No bandpass decided')
                paras = classify_direct.stparas(x, bcmodel=bcmodel, dustmodel=dustmodel, band='jmag', ext=ext, plot=plot)
    
            else:
                x.addmag([photometry_dict[test_band]], [photometry_dict[test_band + 'e']])
                paras = classify_direct.stparas(x, bcmodel=bcmodel, dustmodel=dustmodel, band=test_band, ext=ext, plot=plot)
    
    # Store bolometric luminosity, bolometric magnitudes, and teff
    lum.append(paras.lum)
    lum_err_pos.append(paras.lumep)
    lum_err_neg.append(paras.lumem)
    teff.append(paras.teff)
    mabs.append(paras.mabs)
    
print('\033[1m' + 'Isoclassify successfully run, parameters extracted')
    
# NOTES: Output Interpretation
# Index: The index of data in your FITS file so that it can be referred to later
# No Bandpass Decided: Isoclassify cannot utilize the photometry you have to calculate a distance modulus
# No valid Teff provided or calculated, skipping: Isoclassify does not have enough photometry information 
#                                                 for its color-teff relations to find a teff.

## Print Extracted Parameter Arrays

In [ ]:
# USER: Do not modify contents. Run as is.

# Convert to numpy arrays 
lum = np.array(lum)
teff = np.array(teff)
mabs = np.array(mabs)

# Replace Isoclassify zeros with nan for clarity
lum[np.where(lum == 0)[0]] = np.nan
teff[np.where(teff == 0)[0]] = np.nan

if(print_bolometric_luminosities):
    print('\033[1m'  + 'Bolometric luminosities extracted: ' + '\033[0m')
    print(lum)
    
if(print_bolometric_luminosities_errors):
    print('\033[1m'  + 'Bolometric luminosity upper bound errors extracted: ' + '\033[0m')
    print(lum_err_pos)
    print('\033[1m'  + 'Bolometric luminosity lower bound errors extracted: ' + '\033[0m')
    print(lum_err_neg)
    
if(print_teffs):
    print('\033[1m'  + 'Teffs extracted: ' + '\033[0m')
    print(teff)
    
if(print_mabs):
    print('\033[1m'  + 'Bolometric Magnitudes extracted: ' + '\033[0m')
    print(mabs)
    
# If you are seeing NaN values in the parameter extractions, then not enough data was able for isoclassify to
# provide a result